<a href="https://colab.research.google.com/github/dorigonato/Caes_e_Gatos/blob/main/Caes_e_gatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds

dataset, info = tfds.load('cats_vs_dogs', as_supervised=True, with_info=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.R2OB1U_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [8]:
import zipfile
import os

dataset_path = '/content/PetImages.zip' #define o caminho do dataset

# verifica se o arquivo existe antes de abri-lo
if os.path.exists(dataset_path):
    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall('/content/dataset')  # extrai no diretório
else:
    print(f"Error: '{dataset_path}' not found.")


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/content/dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training')

val_generator = train_datagen.flow_from_directory(
    '/content/dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation')



Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [12]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Carregar o modelo VGG16 pré-treinado
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Adicionar camadas customizadas
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [15]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5)


Epoch 1/5


KeyboardInterrupt: 

In [16]:
model.save('cats_dogs_classifier.h5')


In [17]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Treinamento')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.legend()
plt.show()


NameError: name 'history' is not defined

In [18]:
!git clone https://github.com/dorigonato/Caes_e_Gatos.git


Cloning into 'Caes_e_Gatos'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [20]:
!cp cats_dogs_classifier.h5 /content/Caes_e_Gatos
!cp Caes_e_gatos.ipynb /content/Caes_e_Gatos


cp: cannot stat 'Caes_e_gatos.ipynb': No such file or directory


In [21]:
!cd Caes_e_Gatos && git add .
!cd Caes_e_Gatos && git commit -m "Adicionando projeto de Transfer Learning"
!cd Caes_e_Gatos && git push origin main


Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@2ce2eec104d5.(none)')
fatal: could not read Username for 'https://github.com': No such device or address
